In [ ]:
from pyspark.sql import SparkSession


# create SparkSession
spark = SparkSession.builder \
    .appName("MySQL JDBC Example") \
    .config("spark.jars.packages", "mysql:mysql-connector-java:8.0.33") \
    .config("spark.sql.catalogImplementation", "hive") \
    .config("spark.sql.sources.partitionOverwriteMode", "dynamic") \
    .config("spark.network.timeout", "600s") \
    .getOrCreate()

## read data in Sqoop HDFS

In [ ]:
sqoop_hdfs_path = "hdfs://sqoop-server:9000/user/root/data/sqoop_dictory"

# read the directory as a DataFrame
df_sqoop = spark.read \
    .format("csv") \
    .load(sqoop_hdfs_path)

df_sqoop.printSchema()

## write read data MySQL by spark

In [ ]:
# Configure the MySQL JDBC connection properties
mysql_url = "jdbc:mysql://mysql:3306/your_db"

mysql_properties = {
    "driver": "com.mysql.jdbc.Driver",
    "url": mysql_url,
    "user": "your_user",
    "password": "your_pass",
    "useSSL": "false",
    "autoReconnect": "true",
    "zeroDateTimeBehavior": "convertToNull"
}

# write the DataFrame to the MySQL database
df.write \
    .format("jdbc") \
    .mode("overwrite") \
    .option("dbtable", "your_table") \
    .options(**mysql_properties) \
    .save()

# read the MySQL table to DataFrame
df_mysql = spark.read.jdbc(url=mysql_url, table="your_table", properties=mysql_properties)
df_mysql.count()